Read in data, discard NaN values, then calculate slopes of different neurons, residuals, and deconvolve to get neural activity

In [ ]:
import numpy as np
from scipy.interpolate import interp1d
from scipy.interpolate import interpn
from matplotlib import pyplot as plt
import seaborn as sns
import scipy.io as sio
%pylab inline

# load data
Directory = 'Data/' # for Katherine's computer
#Directory = 'Data/' # for Nick's computer
Ldfile = 'heatData_worm5'
fileread = sio.loadmat(Directory+Ldfile)
afileread

In [ ]:
G2 = np.array(fileread['G2'])
R2 = np.array(fileread['R2'])
ratio = np.array(fileread['R2'])
cgIdx = (np.ravel(np.array(fileread['cgIdx'])) - 1)#[:80]
print(G2.shape)

badcols = np.array([x|y for (x,y) in zip(np.isnan(G2).any(axis=0), np.isnan(R2).any(axis=0))])
intG2 = G2[:,~badcols]
intG2 = intG2[cgIdx]
intR2 = R2[:,~badcols]
intR2 = intR2[cgIdx]

(neurons, time) = intG2.shape
timePoints = np.array(range(1,time))
print(intG2.shape)

f ,ax = plt.subplots(1,2, figsize=(20,5*2))
ax[0].imshow(intG2, aspect=10,cmap="Paired")
ax[1].imshow(intR2, aspect=10,cmap="Paired")
for axis, title in zip(ax, [r'GCAMP', r'RFP']):
    axis.set_title(title)

# Slopes
Now that we have the data read in, let's look at some basic regressions.

In [ ]:
from scipy import stats
import string
X = np.ones((time,1))
Y = np.zeros((time,1))

vals = np.random.randint(0, len(intG2), 10)
f ,ax = plt.subplots(len(vals),1, figsize=(20,len(vals)*10))
for i, v in enumerate(vals):
    Y = intG2[v,:].T
    X = intR2[v,:].T
    slope, intercept, r_value, p_value, std_err = stats.linregress(X,Y)
    ax[i].plot(X,Y, lw=0.2)
    ax[i].scatter(X,Y, c= np.arange(len(X)), cmap='RdBu') # other possible colormaps are: YlOrBr, YlGnBu, 
    ax[i].plot(X, intercept + slope*X)
    ax[i].set_title('Neuron: ' + str(v)+', Slope: '+ "%.3f" % slope + ', R_val: ' + "%.3f" % r_value)

In [ ]:
slopes = np.zeros(neurons)
r_values = np.zeros(neurons)
resids = np.zeros((neurons,time))


for i in range(0,neurons):
    Y = intG2[i,:].T
    X = intR2[i,:].T
    slope, intercept, r_value, p_value, std_err = stats.linregress(X,Y)
    slopes[i] = slope
    r_values[i] = r_value
    
    resids[i,:] = Y-intercept-slope*X

plt.figure(figsize=(20,10))
plt.scatter(range(0,neurons), slopes)
plt.title('Slopes of G2 regressed on R2 for each neuron')

# find that odd negative slope
# import operator
# min_index, min_value = min(enumerate(slopes), key=operator.itemgetter(1))
# print(min_index)

In [ ]:
# rvalues
plt.figure(figsize=(12,8))

plt.scatter(slopes, r_values**2)
plt.title('R^2 values of G2 regressed on R2 for each neuron')
plt.xlabel('Slope')
plt.ylabel('r_Value')
plt.ylim([0,1])


# Residuals
View the residuals

In [ ]:
vals = [65,1,0]
f ,ax = plt.subplots(len(vals),1, figsize=(15,len(vals)*5))

for i, v in enumerate(vals):
    ax[i].plot(np.arange(0,time/6,1/6), resids[i,:])
    ax[i].set_title('Residuals: Neuron: ' + str(v)+', Slope: %.3f' % slope)
    

# Deconvolve
Let's now deconvolve the flouresence to get back our neural activity

In [ ]:
from scipy import signal
import math

alpha = 1.0 # decay rate
decay_time = 50
filt = [math.exp(-alpha*t/6.0) for t in range(0,decay_time)] # divide by time resolution (~6Hz)

print(time - decay_time + 1)

values = [1,3,40,64]
f ,ax = plt.subplots(len(values),1, figsize=(15,4*len(values)))
for i, v in enumerate(values):
    [neural_sig, remainder] = signal.deconvolve(resids[v,:],filt)
    ax[i].plot(np.arange(0,230,1/6), neural_sig[:230*6])
    ax[i].set_title('Neural Signals: Neuron: ' + str(v))
    ax[i].set_xlim([0,230])